# How-to deploy a model on Azure ML

This notebook takes you through the steps of deploying a model on Azure for The Ocean Cleanup. We can deploy any model registered after the training as seen in the notebooks of the `ModelTraining` directory.

Next to the concepts introduced in those notebooks, there are a few additional concepts we need to know about:

- score script: A Python script that contains the code of how to perform inference. Implements two functions:
  - `init()`: Initializing function run once when the container is started. Initialize your model here
  - `run(request)`: Function that is called for every request to the endpoint. This endpoint is called with the
    binary image data as body, which is provided to the function through request.get_data(). The response is
    sent back.
- Service: A deployed model. This provides an `endpoint` to which we can send data.
- Endpoint: This is provided by a service. Is an URI to which inference requests can be sent.
- InferenceConfig: Configuration describing how the inference should be performed. This contains the score script,
  the source it is run in and the environment it should use.
- DeployConfig: The configuration of where to deploy the service. This can be a LocalWebservice, AciWebservice or
  AksWebservice. The former 2 are aimed at development work, the later for production.
- Compute Target: When using an AksWebservice, the Compute Target must be named. This is the AKS cluster where to
  run the service. This should be registered as 'Inference cluster' in Azure ML.
  
Before we get to work with these, we need to load in our workspace again, like before:

In [1]:
from azurewrapper.workspace import get_workspace

subscription_id = "29d66431-a7ce-4709-93f7-3bdb01a243b3"
resource_group = "ExperimentationJayke"
workspace_name = "ExperimentationJayke"

workspace = get_workspace(subscription_id, resource_group, workspace_name)

Now, like before, we need to create or load an environment:

In [2]:
from azurewrapper.environment import get_environment

environment = get_environment(
    workspace,
    "example_model_env",
    pip_requirements="./examples/example_model/requirements.txt"
)

No environment with that name found, creating new one


Now it's time to load the model. Note that it is possible to provide multiple models to the Service. However, in the current architectural design, we deploy a single model as a single service, and therefore single endpoint.

In [3]:
from azureml.core import Model

model = Model(workspace, "example_model")

At this time, the score script should be implemented. The score script implements an `init()` function, and a `run(request)` function. Note that we will deploy the entire folder containing the `score.py` script, so it is perfectly fine to use imports within that and spread your code over multiple files.

The provided model is used as follows: The artifacts stored with the model will be loaded into a folder, that is pointed to by the environment variable `AZUREML_MODEL_DIR`. When multiple models are provided, there will be an extra layer of subdirectories there, where each folder has the name of a model used. The artifacts will then be inside that folder.

The folder structure is kept the same as shown in the `Artifacts` tab of the Azure ML Studio interface for a model.

## Deploying the model

Once we have the score script ready, it's time to deploy the model. The function `azurewrapper.deploy.deploy()` can help with this. It will create the correct InferenceConfig and DeployConfig, and will use those to create the service.

In [17]:
from azurewrapper.deploy import deploy
service = deploy(
    workspace,
    "testdeploy",
    model,
    "score.py",
    "examples/example_model",
    environment=environment,
    target='aks',
    compute_target_name='deploy-test'
)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......
Succeeded
AKS service creation operation finished, operation "Succeeded"


Once the service is created, we can determine the scoring uri. This (naturally) depends on the type of service that was created. The following possible endpoints will be created:
- LocalWebservice: `http://localhost:8890/score` (port is defined in `azurewrapper.deploy.deploy()`
- AciWebservice: `http://<some uuid>.<region>.azurecontainer.io/score`,
  eg `http://5222e98a-6791-4ed8-a4c6-c44e36b0c9cb.westeurope.azurecontainer.io/score`
- AksWebservice: `http://<cluster IP>/api/v1/service/<service name>/score`,
  eg. `http://20.71.137.87:80/api/v1/service/testdeploy/score`
    
Other interesting endpoints may be the same url without `/score` (eg `http://20.71.137.87:80/api/v1/service/testdeploy`, this provides a health check, and `http://20.71.137.87:80/api/v1/service/testdeploy/ui` for a Swagger interface of the API.

In [18]:
service.scoring_uri

'http://13.95.2.180:80/api/v1/service/testdeploy/score'

In [15]:
service.get_logs()

'2020-11-11T15:10:20,532805734+00:00 - rsyslog/run \n2020-11-11T15:10:20,533104136+00:00 - iot-server/run \n/usr/sbin/nginx: /azureml-envs/azureml_6cfb08ae90424295b381c464fea452e2/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_6cfb08ae90424295b381c464fea452e2/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_6cfb08ae90424295b381c464fea452e2/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_6cfb08ae90424295b381c464fea452e2/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_6cfb08ae90424295b381c464fea452e2/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-11-11T15:10:20,535227350+00:00 - nginx/run \n2020-11-11T15:10:20,537662666+00:00 - gunicorn/run 

In [16]:
service.delete()